In [1]:
import multiprocessing
import numpy as np
import pandas as pd
import os
import transforms3d as tf

In [2]:
data_dir = "indoor_forward_9_davis_with_gt"
events_cols=["timestamp", "x", "y", "polarity"]
events_df = pd.read_csv(os.path.join(data_dir,"events.txt"), delimiter=' ', skiprows=1, names=events_cols)

In [3]:
label_cols=["timestamp","tx","ty","tz","qx","qy","qz","qw","none"]
labels_df = pd.read_csv(os.path.join(data_dir,"groundtruth.txt"), delimiter=' ', skiprows=1, names=label_cols)
labels_df = labels_df.iloc[: , :-1]
labels_df["events_start_idx"] = 0
labels_df_nostamp = labels_df.loc[:, labels_df.columns != 'timestamp']
labels_df_nostamp = labels_df_nostamp.loc[:, labels_df_nostamp.columns != 'events_start_idx']
labels = labels_df_nostamp.to_numpy()
labels_df

,timestamp,tx,ty,tz,qx,qy,qz,qw,events_start_idx
0,1.540823e+09,6.972598,3.387750,-0.847228,-0.226137,-0.671406,0.663737,0.239853,0
1,1.540823e+09,6.972586,3.387697,-0.847242,-0.226178,-0.671366,0.663755,0.239874,0
2,1.540823e+09,6.972573,3.387645,-0.847256,-0.226212,-0.671332,0.663772,0.239894,0
3,1.540823e+09,6.972558,3.387596,-0.847270,-0.226238,-0.671302,0.663786,0.239912,0
4,1.540823e+09,6.972541,3.387547,-0.847284,-0.226257,-0.671278,0.663798,0.239930,0
...,...,...,...,...,...,...,...,...,...
14395,1.540823e+09,12.768051,-0.781329,0.868929,-0.820637,0.224594,-0.161882,0.499907,0
14396,1.540823e+09,12.775141,-0.767458,0.868216,-0.820292,0.225228,-0.161738,0.500233,0
14397,1.540823e+09,12.782237,-0.753578,0.867507,-0.819948,0.225858,-0.161598,0.500560,0
14398,1.540823e+09,12.789339,-0.739690,0.866803,-0.819603,0.226485,-0.161461,0.500886,0


In [14]:
new_cols = ["dtx","dty","dtz","dqw","dqx","dqy","dqz"]
for col in new_cols:
    labels_df[col] = 0.0
first_id = abs(events_df["timestamp"]-labels_df.iloc[0]["timestamp"]).idxmin()
threshold = 1e-2 # 100 Hz
delta = 10
for i in range(len(labels_df)-delta):
    label_ts = labels_df.iloc[i,0]
    # hopefully no more than 10k event between labels
    id = events_df[first_id:first_id+int(1e4)]["timestamp"].sub(label_ts).abs().idxmin()
    labels_df["events_start_idx"][i] = id
    first_id = id
    if i % 1000 == 0:
        print(i)

    dpose = np.zeros(7, dtype=np.float64)
    l1 = labels[i]
    l2 = labels[i+delta]
    dl = l2-l1 # position delta
    dpose[:3] = dl[:3]
    q1 = l1[3:][[3,0,1,2]] # rearange quaternion, w goes first
    q2 = l2[3:][[3,0,1,2]] # rearange quaternion, w goes first
    dq = tf.quaternions.qmult(q2, tf.quaternions.qinverse(q1)) # orientation delta
    dpose[3:] = dq # pose delta is the label

    for col_index,col in enumerate(new_cols):
        labels_df[col][i+delta] = dpose[col_index]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [15]:
prev_labels_df = pd.read_pickle("event_indexed_labels.pickle")
(prev_labels_df["events_start_idx"] - labels_df["events_start_idx"]).sum() # safety check, should be zero

0

In [19]:
labels_df[100:200]

,timestamp,tx,ty,tz,qx,qy,qz,qw,events_start_idx,dtx,dty,dtz,dqw,dqx,dqy,dqz
100,1.540823e+09,6.970961,3.383977,-0.848637,-0.226065,-0.670974,0.663952,0.240536,6542119,-0.000066,-0.000314,-0.000083,1.0,0.000023,0.000082,5.527568e-05
101,1.540823e+09,6.970956,3.383946,-0.848643,-0.226056,-0.670974,0.663954,0.240539,6542121,-0.000064,-0.000313,-0.000079,1.0,0.000022,0.000082,5.394097e-05
102,1.540823e+09,6.970950,3.383915,-0.848648,-0.226047,-0.670973,0.663956,0.240542,6542124,-0.000061,-0.000313,-0.000075,1.0,0.000020,0.000083,5.232951e-05
103,1.540823e+09,6.970946,3.383884,-0.848653,-0.226038,-0.670973,0.663958,0.240545,6542137,-0.000059,-0.000312,-0.000070,1.0,0.000019,0.000083,5.045626e-05
104,1.540823e+09,6.970941,3.383853,-0.848658,-0.226030,-0.670972,0.663960,0.240548,6542141,-0.000057,-0.000311,-0.000066,1.0,0.000017,0.000084,4.833731e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1.540823e+09,6.971208,3.381359,-0.848790,-0.226060,-0.670757,0.664032,0.240921,6542292,0.000054,-0.000208,0.000067,1.0,0.000018,0.000027,-3.908625e-06
196,1.540823e+09,6.971213,3.381340,-0.848779,-0.226057,-0.670757,0.664032,0.240923,6542292,0.000053,-0.000205,0.000074,1.0,0.000018,0.000029,-2.615843e-06
197,1.540823e+09,6.971217,3.381321,-0.848767,-0.226054,-0.670757,0.664032,0.240926,6542294,0.000051,-0.000203,0.000082,1.0,0.000017,0.000030,-1.359451e-06
198,1.540823e+09,6.971222,3.381302,-0.848755,-0.226051,-0.670757,0.664033,0.240929,6542296,0.000050,-0.000200,0.000089,1.0,0.000016,0.000032,-1.392812e-07


In [17]:
labels_df.to_pickle("event_indexed_labels.pickle")

In [18]:
# from PIL import Image
# import matplotlib.pyplot as plt
# save_dir = os.path.join(data_dir, "events")
# if not(os.path.exists(save_dir) and os.path.isdir(save_dir)):
#     os.mkdir(save_dir)
# events_df_nostamp = events_df.loc[:, events_df.columns != 'timestamp']
# events_data = events_df_nostamp.to_numpy()
# bins = 3
# mc = 350 # 345 is max dim
# delta = 25 # corresponding to running loc at 10Hz
# for idx in range(len(labels_df)):
#     start = labels_df.iloc[idx]["events_start_idx"]
#     finish = labels_df.iloc[idx+delta]["events_start_idx"]
#     num_per_bin = int((finish-start)/bins)
#     img = np.zeros((bins,mc,mc), dtype=np.float32)
#     for b in range(bins):
#         start_bin = int(start)+b*num_per_bin
#         if b != (bins - 1):
#             events = np.array(events_data[start_bin:start_bin+num_per_bin], dtype=np.uint16)
#         else:
#             events = np.array(events_data[start_bin:int(finish)], dtype=np.uint16)
#         np.add.at(img[b],tuple([events[:,1], events[:,0]]), events[:,2] - .5)
#     np.save(os.path.join(data_dir, "events", f"{idx}.npy"), img)